In [1]:
# -*-encoding=utf-8-*-
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np


In [27]:
# itemdata=pd.read_csv("C:\Users\scc\PycharmProjects\untitled1\datadig2\\tianchi_fresh_comp_train_item.csv")
# userdata=pd.read_csv("C:\Users\scc\PycharmProjects\untitled1\datadig2\\new_train_user.csv")
itemdata=pd.read_csv("D:\DAUM\\tianchi_fresh_comp_train_item.csv")
userdata=pd.read_csv("D:\DAUM\\new_train_user.csv")

In [28]:
#取出唯一的商品ID和类别
itemdata_id_category=itemdata.drop_duplicates(['item_id','item_category'])
print itemdata_id_category.shape

(422858, 3)


In [29]:
#取出有交互的商品数目
sub=pd.merge(itemdata_id_category,userdata,how='inner',on=['item_id','item_category'])
print sub.shape


(2084859, 7)


In [30]:
# print sub.head(10)
cols = list(sub)
print cols
cols=['user_id', 'item_id', 'item_geohash', 'item_category', 'behavior_type', 'date', 'hour']
df = sub.ix[:, cols]
del df['item_geohash']
print df.head(10)

['item_id', 'item_geohash', 'item_category', 'user_id', 'behavior_type', 'date', 'hour']


     user_id    item_id  item_category  behavior_type        date  hour
0   20723532  100002303           3368              1  2014-11-22    18
1   20723532  100002303           3368              1  2014-11-22    18
2   20723532  100002303           3368              1  2014-11-22    18
3   44416664  100002303           3368              1  2014-12-16    15
4   44416664  100002303           3368              1  2014-12-16    15
5  142281897  100002303           3368              1  2014-12-02    23
6  109084811  100003592           7995              1  2014-11-24    18
7  134325108  100006838          12630              1  2014-11-19    21
8    5015420  100008089           7791              1  2014-12-09    20
9  135142209  100012750           9614              1  2014-11-22    23


In [31]:
#选取17号作为特征数据
feature_date = '2014-12-17'
df3 = df[(df.date == feature_date) & (df.behavior_type == 3) & (df.hour >= 18)]
print df3.shape
print df3.head(10)


(768, 6)
         user_id    item_id  item_category  behavior_type        date  hour
127    132236660  100079162           4778              3  2014-12-17    20
6406   116631182  103835557           1483              3  2014-12-17    18
9890    25730704  105318396           9232              3  2014-12-17    18
14301  108324380  107852873           3942              3  2014-12-17    20
21671   39175121  111602834          11991              3  2014-12-17    19
22758  135378246  112189655          11497              3  2014-12-17    22
32377  132626507   11685888           3064              3  2014-12-17    21
34330   13855326  117807825           7876              3  2014-12-17    22
38568   36548990  120063996          11991              3  2014-12-17    21
42320   17562880  122174462           4245              3  2014-12-17    22


In [32]:
df4=df[(df.date == feature_date) & (df.behavior_type == 4) & (df.hour >= 18)]
print df4.shape
print df4.head(10)

(297, 6)
         user_id    item_id  item_category  behavior_type        date  hour
1025    42369712  100570822           3956              4  2014-12-17    22
4147   113190941  102360256          13547              4  2014-12-17    21
5200    33541423  102984080          10015              4  2014-12-17    18
29841   37165541   11572504           6648              4  2014-12-17    22
38776   28674134  120163823           5324              4  2014-12-17    22
57431   10874175  129098469           7791              4  2014-12-17    20
58409   45470234  129662365           1963              4  2014-12-17    22
64056    4564376  132929824           1686              4  2014-12-17    21
64103   49953924  132968128           9232              4  2014-12-17    20
69589   58404006  135655048           6164              4  2014-12-17    22


In [33]:
#17日当天加购并且购买
addandbuy=pd.merge(df3,df4,how='inner',on=['user_id','item_id'])
addandbuy['addandbuy']=1
print addandbuy.shape
print addandbuy.head(10)

(147, 11)
     user_id    item_id  item_category_x  behavior_type_x      date_x  hour_x  \
0    4564376  132929824             1686                3  2014-12-17      21   
1   49953924  132968128             9232                3  2014-12-17      20   
2   58404006  135655048             6164                3  2014-12-17      22   
3   40513258  140944547             3368                3  2014-12-17      20   
4   40513258  140944547             3368                3  2014-12-17      20   
5   55540832  155726599             3505                3  2014-12-17      18   
6  113903693  156595505             2158                3  2014-12-17      21   
7  113903693  156595505             2158                3  2014-12-17      21   
8  118630104  176211790             5543                3  2014-12-17      20   
9    5311010  184420640             2284                3  2014-12-17      18   

   item_category_y  behavior_type_y      date_y  hour_y  addandbuy  
0             1686           

In [34]:
#18点是否加购的特征加入df3
df_18_ifadd=pd.merge(df3,addandbuy,how='left',on=['user_id','item_id'])
df_18_ifadd=df_18_ifadd.fillna(0)
print df_18_ifadd.shape
print df_18_ifadd.head(1)

(801, 15)
     user_id    item_id  item_category  behavior_type        date  hour  \
0  132236660  100079162           4778              3  2014-12-17    20   

   item_category_x  behavior_type_x date_x  hour_x  item_category_y  \
0              0.0              0.0      0     0.0              0.0   

   behavior_type_y date_y  hour_y  addandbuy  
0              0.0      0     0.0        0.0  


In [35]:
# 浏览量统计
df_brows_count = df[(df.date == feature_date) & (df.behavior_type == 1)]
df_brows_count = df_brows_count.groupby(['user_id', 'item_id'], as_index=False) \
    .behavior_type.agg({'brows_count': np.size})
print df_brows_count.shape
print df_brows_count.head(10)

(29662, 3)
   user_id    item_id  brows_count
0    59436  238861461            2
1    60723  202829025            2
2    60723  371933634            2
3   106362   38830684            1
4   106362  149517272            2
5   106362  200051509            1
6   106362  236862264            1
7   106362  308735138            1
8   106362  342598663            1
9   106362  364101379            1


In [36]:
# 收藏量统计
df_collect_count = df[(df.date == feature_date) & (df.behavior_type == 2)]
df_collect_count = df_collect_count.groupby(['user_id', 'item_id'], as_index=False) \
    .behavior_type.agg({'collect_count': np.size})
print df_collect_count.shape
print df_collect_count.head(10)


(1132, 3)
   user_id    item_id  collect_count
0   487386  135995541              1
1   487386  396534177              1
2   595845   33619451              1
3  1031395  127516527              1
4  1484968  105857507              1
5  1884205  170407016              1
6  1996141  181327497              1
7  2046058   15604509              1
8  2090599  101403637              1
9  2090599  296017233              1


In [37]:
# 加购量统计
df_add_count = df[(df.date == feature_date) & (df.behavior_type == 3)]
df_add_count = df_add_count.groupby(['user_id', 'item_id'], as_index=False) \
    .behavior_type.agg({'add_count': np.size})
print df_add_count.shape
print df_add_count.head(10)

(1724, 3)
   user_id    item_id  add_count
0   161481   14285814          1
1   161481   25651335          1
2   161481  155199637          1
3   161481  235875168          1
4   161481  383479841          1
5   226017    9592306          1
6  1012927  105857507          1
7  1012927  216810080          1
8  1096774   57739007          1
9  1107934  298268624          1


In [38]:
df_buy_count = df[(df.date == feature_date) & (df.behavior_type == 4)]
df_buy_count = df_buy_count.groupby(['user_id', 'item_id'], as_index=False) \
    .behavior_type.agg({'buy_count': np.size})
print df_buy_count.shape
print df_buy_count.head(10)

(675, 3)
   user_id    item_id  buy_count
0  1096774   57739007          1
1  1105255   22302304          1
2  1285111  264035656          1
3  1287220  148781276          1
4  1675813  202021439          1
5  1675813  382820731          1
6  1694224  340480775          1
7  1945207  385130797          1
8  1996141  304121817          1
9  2292421  181630953          1


In [39]:
# 合并字段
tezhengdf = df3.loc[:, ['user_id', 'item_id','item_category', 'addandbuy']]
tmpdf1=pd.merge(df_brows_count,df_collect_count,how='outer',on=['user_id','item_id'])
tmpdf2=pd.merge(df_add_count,df_buy_count,how='outer',on=['user_id','item_id'])
tmpdf3=pd.merge(tmpdf1,tmpdf2,how='outer',on=['user_id','item_id'])
feature_df=pd.merge(tmpdf3,tezhengdf,how='outer',on=['user_id','item_id'])
feature_df=feature_df.fillna(0)
print feature_df.shape
print feature_df.head()


(29761, 8)
    user_id      item_id  brows_count  collect_count  add_count  buy_count  \
0   59436.0  238861461.0          2.0            0.0        0.0        0.0   
1   60723.0  202829025.0          2.0            0.0        0.0        0.0   
2   60723.0  371933634.0          2.0            0.0        0.0        0.0   
3  106362.0   38830684.0          1.0            0.0        0.0        0.0   
4  106362.0  149517272.0          2.0            0.0        0.0        0.0   

   item_category  addandbuy  
0            0.0        0.0  
1            0.0        0.0  
2            0.0        0.0  
3            0.0        0.0  
4            0.0        0.0  


In [40]:
# 购买占浏览量的比例
feature_df['buyofbrows'] = feature_df['buy_count']/feature_df['brows_count']
# 购买占收藏的比例
feature_df['buyofcollect'] = feature_df['buy_count']/feature_df['collect_count']
# 购买占加购的比例
feature_df['buyofadd'] = feature_df['buy_count']/feature_df['add_count']
print feature_df.head(20)


     user_id      item_id  brows_count  collect_count  add_count  buy_count  \
0    59436.0  238861461.0          2.0            0.0        0.0        0.0   
1    60723.0  202829025.0          2.0            0.0        0.0        0.0   
2    60723.0  371933634.0          2.0            0.0        0.0        0.0   
3   106362.0   38830684.0          1.0            0.0        0.0        0.0   
4   106362.0  149517272.0          2.0            0.0        0.0        0.0   
5   106362.0  200051509.0          1.0            0.0        0.0        0.0   
6   106362.0  236862264.0          1.0            0.0        0.0        0.0   
7   106362.0  308735138.0          1.0            0.0        0.0        0.0   
8   106362.0  342598663.0          1.0            0.0        0.0        0.0   
9   106362.0  364101379.0          1.0            0.0        0.0        0.0   
10  106362.0  377408757.0          1.0            0.0        0.0        0.0   
11  137907.0  166251701.0          4.0            0.

In [41]:
#提取特征最近1天（用户）的浏览数量
u_df1 = df[(df.date == feature_date) & (df.behavior_type == 1)]
u_df1 = u_df1.groupby(['user_id'], as_index=False)['behavior_type'].agg({'u_brows_count': np.size})
print u_df1.head()

   user_id  u_brows_count
0    59436              2
1    60723              4
2   106362              9
3   137907              4
4   161481             20


In [42]:
#提取特征最近1天（用户）的收藏的记录数量
u_df2=df[(df.date==feature_date) & (df.behavior_type==2)]
u_df2 =u_df2.groupby(['user_id'], as_index=False)['behavior_type'].agg({'u_collect_count': np.size})
print u_df2.head()

   user_id  u_collect_count
0   487386                2
1   595845                1
2  1031395                1
3  1484968                1
4  1884205                1


In [43]:
#提取特征最近1天（用户）的加购的记录数量
u_df3=df[(df.date==feature_date) & (df.behavior_type==3)]
u_df3 =df3.groupby(['user_id'], as_index=False)['user_id'].agg({'u_add_count': np.size})
print u_df3.head()

   user_id  u_add_count
0   226017            1
1  1136287            4
2  2137246            2
3  2437852            1
4  2631448            1


In [44]:
#提取特征最近1天（用户）的购买的记录数量
u_df4=df[(df.date==feature_date) & (df.behavior_type==4)]
u_df4 =df4.groupby(['user_id'], as_index=False)['user_id'].agg({'u_buy_count': np.size})
print u_df4.head()

   user_id  u_buy_count
0  1105255            1
1  1285111            1
2  1945207            1
3  1996141            1
4  2292421            1


In [45]:
#将用户特征进行合并
u_feature_df=pd.merge(u_df1,u_df2,how='outer',on=['user_id'])
u_feature_df1=pd.merge(u_df3,u_df4,how='outer',on=['user_id'])
u_feature_df=pd.merge(u_feature_df,u_feature_df1,how='outer',on=['user_id'])
u_feature_df=u_feature_df.fillna(0)
print u_feature_df.shape,u_feature_df.head()

(4130, 5)     user_id  u_brows_count  u_collect_count  u_add_count  u_buy_count
0   59436.0            2.0              0.0          0.0          0.0
1   60723.0            4.0              0.0          0.0          0.0
2  106362.0            9.0              0.0          0.0          0.0
3  137907.0            4.0              0.0          0.0          0.0
4  161481.0           20.0              0.0          0.0          0.0


In [46]:
#将特征合并
feature_df=pd.merge(feature_df,u_feature_df,how='left',on=['user_id'])
feature_df=feature_df.fillna(0)
print feature_df.shape,feature_df.head(20)

(29761, 15)      user_id      item_id  brows_count  collect_count  add_count  buy_count  \
0    59436.0  238861461.0          2.0            0.0        0.0        0.0   
1    60723.0  202829025.0          2.0            0.0        0.0        0.0   
2    60723.0  371933634.0          2.0            0.0        0.0        0.0   
3   106362.0   38830684.0          1.0            0.0        0.0        0.0   
4   106362.0  149517272.0          2.0            0.0        0.0        0.0   
5   106362.0  200051509.0          1.0            0.0        0.0        0.0   
6   106362.0  236862264.0          1.0            0.0        0.0        0.0   
7   106362.0  308735138.0          1.0            0.0        0.0        0.0   
8   106362.0  342598663.0          1.0            0.0        0.0        0.0   
9   106362.0  364101379.0          1.0            0.0        0.0        0.0   
10  106362.0  377408757.0          1.0            0.0        0.0        0.0   
11  137907.0  166251701.0          4.0  

In [47]:
#取18号数据
target_df = df[(df.date == '2014-12-18') & (df.behavior_type == 4)]
target_df['target']=1
target_df=target_df.loc[:,['user_id','item_id','target']]
print target_df.shape
print target_df.head(20)


(781, 3)
         user_id    item_id  target
2017     4860935  101081429       1
3895    16476718  102217808       1
4762    16401055  102716055       1
8492   137515335  104821990       1
9565   138589540  105209861       1
10345   48594900  105573580       1
11704   12073638  106304028       1
13292   10634751  107142845       1
16657  110948224  109229077       1
20652  131395085  111161394       1
20744   32840272  111163483       1
22761  135378246  112189655       1
22825   18249259  112214392       1
31557  132755942   11653679       1
32488  136948968  116886397       1
34671  124299848  117991581       1
37226  122688869  119315168       1
44043  103490614  122771636       1
45048  122357858  123339153       1
46224  113579957  123601142       1


D:\Anaconda2\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [48]:
feature_data=pd.merge(feature_df,target_df,how='left',on=['user_id','item_id']).drop_duplicates(['user_id','item_id'])
print feature_data.shape
feature_data=feature_data.fillna(0)
feature_data=pd.DataFrame.round(feature_data,2)
print feature_data.head(10)

(29749, 16)
    user_id      item_id  brows_count  collect_count  add_count  buy_count  \
0   59436.0  238861461.0          2.0            0.0        0.0        0.0   
1   60723.0  202829025.0          2.0            0.0        0.0        0.0   
2   60723.0  371933634.0          2.0            0.0        0.0        0.0   
3  106362.0   38830684.0          1.0            0.0        0.0        0.0   
4  106362.0  149517272.0          2.0            0.0        0.0        0.0   
5  106362.0  200051509.0          1.0            0.0        0.0        0.0   
6  106362.0  236862264.0          1.0            0.0        0.0        0.0   
7  106362.0  308735138.0          1.0            0.0        0.0        0.0   
8  106362.0  342598663.0          1.0            0.0        0.0        0.0   
9  106362.0  364101379.0          1.0            0.0        0.0        0.0   

   item_category  addandbuy  buyofbrows  buyofcollect  buyofadd  \
0            0.0        0.0         0.0           0.0       0.

In [49]:
feature_data.to_csv('D:\DAUM\\test.csv',index=False)